In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mp
import matplotlib.dates as md
import matplotlib.pyplot as plt
import os
import math
import tensorflow as tf
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense,Flatten,Activation
from keras.layers import LSTM
from keras.optimizers import Adam,RMSprop
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping
from keras.layers import Conv1D, Conv2D, MaxPooling2D,MaxPooling1D

In [ ]:
EGX30 = pd.read_csv('final.csv')

In [ ]:
#Original data
EGX30

,Date,Close,Open,High,Low,Volume,Change %
0,5/2/2024,"26,113.71","25,917.59","26,168.25","25,190.92",189.62M,3.34%
1,5/1/2024,"25,270.55","25,917.59","25,294.43","24,195.27",182.43M,3.36%
2,4/30/2024,"24,448.73","26,011.28","26,097.30","24,409.83",264.22M,-6.01%
3,4/29/2024,"26,011.28","24,639.63","26,011.28","24,637.16",220.16M,5.57%
4,4/28/2024,"24,639.63","25,917.59","26,005.67","24,639.63",208.46M,-4.93%
...,...,...,...,...,...,...,...
3682,3/5/2009,"3,575.93","3,566.55","3,605.23","3,565.60",54.81M,0.47%
3683,3/4/2009,"3,559.09","3,524.63","3,562.48","3,524.63",68.90M,1.19%
3684,3/3/2009,"3,517.33","3,524.81","3,533.41","3,494.95",57.01M,-1.88%
3685,3/2/2009,"3,584.86","3,640.55","3,641.19","3,584.86",46.91M,-1.70%


In [ ]:
EGX30.set_index('Date', inplace=True)
EGX30 = EGX30.dropna()

In [ ]:
dataframe = EGX30

In [ ]:
#Sorting the data from old to new
dataframe = dataframe[::-1]

In [ ]:
dataframe

,Close,Open,High,Low,Volume,Change %
Date,,,,,,
3/1/2009,"3,647.02","3,597.56","3,648.15","3,588.01",43.85M,1.37%
3/2/2009,"3,584.86","3,640.55","3,641.19","3,584.86",46.91M,-1.70%
3/3/2009,"3,517.33","3,524.81","3,533.41","3,494.95",57.01M,-1.88%
3/4/2009,"3,559.09","3,524.63","3,562.48","3,524.63",68.90M,1.19%
3/5/2009,"3,575.93","3,566.55","3,605.23","3,565.60",54.81M,0.47%
...,...,...,...,...,...,...
4/28/2024,"24,639.63","25,917.59","26,005.67","24,639.63",208.46M,-4.93%
4/29/2024,"26,011.28","24,639.63","26,011.28","24,637.16",220.16M,5.57%
4/30/2024,"24,448.73","26,011.28","26,097.30","24,409.83",264.22M,-6.01%


In [ ]:
#Modifying the index to follow the right pattern.
dataframe.drop(dataframe.columns.difference(['Close']), axis=1, inplace=True)

<ipython-input-8-ecaed88d030d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.drop(dataframe.columns.difference(['Close']), axis=1, inplace=True)


In [ ]:
dataframe['Close'] = dataframe['Close'].str.replace(',', '')

<ipython-input-9-ee6d52056bbc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Close'] = dataframe['Close'].str.replace(',', '')


In [ ]:
dataframe

,Close
Date,
3/1/2009,3647.02
3/2/2009,3584.86
3/3/2009,3517.33
3/4/2009,3559.09
3/5/2009,3575.93
...,...
4/28/2024,24639.63
4/29/2024,26011.28
4/30/2024,24448.73


In [ ]:
# fix random seed for reproducibility
np.random.seed(42)

In [ ]:
# normalize the dataset
dataset = dataframe.values

In [ ]:
dataset

array([['3647.02'],
       ['3584.86'],
       ['3517.33'],
       ...,
       ['24448.73'],
       ['25270.55'],
       ['26113.71']], dtype=object)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
dataset.shape

(3660, 1)

In [ ]:
print(len(dataset) * 0.1)

366.0


In [ ]:
#Splitting the data to be 80% training and 10% for testing
print(len(dataset))
train_size = int(len(dataset)-366)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

3660
3294 366


In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
# reshape into X=t and Y=t+1
look_back = 2
X_train, y_train = create_dataset(train, look_back)
X_test, y_test = create_dataset(test, look_back)

In [ ]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
X_train.shape

(3291, 1, 2)

In [ ]:
X_test.shape

(363, 1, 2)

In [ ]:
print(X_train.shape)
x_train2 = X_train.reshape((3291,2,1))
print(x_train2.shape)
print(y_train.shape)
print(X_test.shape)
x_test2 = X_test.reshape((363,2,1))

(3291, 1, 2)
(3291, 2, 1)
(3291,)
(363, 1, 2)


In [ ]:
#Now creating the model.
cnn = Sequential()
cnn.add(Conv1D(4, kernel_size=2, strides=1,padding='same',
                 activation='relu',
                 input_shape=(2,1)))
cnn.add(MaxPooling1D(pool_size=2))

cnn.add(Flatten())
cnn.add(Dense(30))

cnn.add(Dense(1))
cnn.summary()
cnn.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')
checkpointer = ModelCheckpoint(filepath="best_weights_cnn.hdf5", verbose=0, save_best_only=True) # save best model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2, 4)              12        
                                                                 
 max_pooling1d (MaxPooling1  (None, 1, 4)              0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 30)                150       
                                                                 
 dense_1 (Dense)             (None, 1)                 31        
                                                                 
Total params: 193 (772.00 Byte)
Trainable params: 193 (772.00 Byte)
Non-trainable params: 0 (0.00 Byte)
__________________

In [ ]:
cnn.fit(x_train2,y_train, batch_size=2,verbose=2,epochs=100)
# cnn.fit(x_train2, y_train, validation_data=(x_test2, y_test), callbacks=[monitor,checkpointer],verbose=2, epochs=25)

Epoch 1/100
1646/1646 - 11s - loss: 4.4399e-04 - accuracy: 3.0386e-04 - 11s/epoch - 7ms/step
Epoch 2/100
1646/1646 - 7s - loss: 3.3378e-05 - accuracy: 3.0386e-04 - 7s/epoch - 4ms/step
Epoch 3/100
1646/1646 - 6s - loss: 3.4163e-05 - accuracy: 3.0386e-04 - 6s/epoch - 3ms/step
Epoch 4/100
1646/1646 - 4s - loss: 3.6670e-05 - accuracy: 3.0386e-04 - 4s/epoch - 2ms/step
Epoch 5/100
1646/1646 - 4s - loss: 3.5466e-05 - accuracy: 3.0386e-04 - 4s/epoch - 2ms/step
Epoch 6/100
1646/1646 - 5s - loss: 3.8287e-05 - accuracy: 3.0386e-04 - 5s/epoch - 3ms/step
Epoch 7/100
1646/1646 - 4s - loss: 3.2040e-05 - accuracy: 3.0386e-04 - 4s/epoch - 2ms/step
Epoch 8/100
1646/1646 - 4s - loss: 3.3252e-05 - accuracy: 3.0386e-04 - 4s/epoch - 2ms/step
Epoch 9/100
1646/1646 - 5s - loss: 3.0624e-05 - accuracy: 3.0386e-04 - 5s/epoch - 3ms/step
Epoch 10/100
1646/1646 - 4s - loss: 3.2954e-05 - accuracy: 3.0386e-04 - 4s/epoch - 2ms/step
Epoch 11/100
1646/1646 - 6s - loss: 2.8749e-05 - accuracy: 3.0386e-04 - 6s/epoch - 4ms/

In [ ]:
from sklearn.metrics import mean_squared_error
trainPredict = cnn.predict(x_train2)
testPredict = cnn.predict(x_test2)
# invert predictions
# print(trainPredict)
trainPredict = scaler.inverse_transform(trainPredict)
print(trainPredict)
y_train = scaler.inverse_transform([y_train])
testPredict = scaler.inverse_transform(testPredict)
y_test = scaler.inverse_transform([y_test])
# calculate root mean squared error

12/12 [==============================] - 0s 5ms/step
[[ 3667.6313]
 [ 3667.6313]
 [ 3667.6313]
 ...
 [10445.374 ]
 [10515.358 ]
 [11077.0205]]


In [ ]:
trainScore = math.sqrt(mean_squared_error(y_train[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))

Train Score: 131.30 RMSE


##Converting the model to tensor lite

In [ ]:
#For saving the model for later
# (to generate a SavedModel) tf.saved_model.save(model, "saved_model_keras_dir")

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(cnn)
tflite_model = converter.convert()

# Save the model.
with open('CNNmodel.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
cnn.predict([[[0.0],[0.00056041]]])
